# Network Metrics

Build the network graph, compute centrality metrics, detect communities, and save outputs to `data/processed/`

## Setup
Import helpers / ensure the project `src` directory is on the path

In [3]:
import os, sys
# Resolve project root as parent of this notebook's directory
NOTEBOOK_DIR = os.path.abspath(os.getcwd())
PROJECT_ROOT = os.path.abspath(os.path.join(NOTEBOOK_DIR, os.pardir))
SRC_DIR = os.path.join(PROJECT_ROOT, "src")
if SRC_DIR not in sys.path:
    sys.path.append(SRC_DIR)

from build_network import load_and_build
from centrality_analysis import compute_centrality_measures, save_centrality_rankings
from community_detection import detect_communities, save_communities

print(f"Project root: {PROJECT_ROOT}")

Project root: /Users/masoncacurak/Downloads/CS_5483/dallas_network_analysis


## Build graph
Load processed data and build a directed graph using congested travel times

In [5]:
G = load_and_build(weight_type="congested")
print(f"Graph built: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

Loading processed_nodes.csv...
Loading processed_links.csv...
Building NetworkX graph using congested weight...
Adding 21389 nodes...
Adding 35696 edges...
Graph build complete: 21389 nodes, 35696 edges
Graph built: 21389 nodes, 35696 edges


## Centrality metrics
Compute degree, betweenness (weighted), and eigenvector centrality --> save rankings to CSV

In [7]:
centrality = compute_centrality_measures(G)
centrality_path = save_centrality_rankings(centrality)
print(f"Centrality rankings saved to: {centrality_path}")

Computing degree centrality...

Top 10 nodes by degree centrality:
 1. Node 757: 0.000468
 2. Node 11593: 0.000468
 3. Node 1287: 0.000421
 4. Node 107: 0.000374
 5. Node 108: 0.000374
 6. Node 126: 0.000374
 7. Node 176: 0.000374
 8. Node 359: 0.000374
 9. Node 402: 0.000374
10. Node 470: 0.000374
Degree stats -> min: 0.000000, max: 0.000468, mean: 0.000156

Computing betweenness centrality by weight...

Top 10 nodes by betweenness centrality:
 1. Node 19147: 0.234273
 2. Node 1293: 0.222098
 3. Node 15747: 0.215056
 4. Node 19419: 0.215056
 5. Node 2209: 0.209543
 6. Node 3472: 0.209543
 7. Node 12164: 0.206209
 8. Node 15771: 0.205407
 9. Node 11589: 0.199713
10. Node 11546: 0.199713
Betweenness stats -> min: 0.000000, max: 0.234273, mean: 0.022369

Computing eigenvector centrality on largest connected component...
Using strongly connected component with 20434 nodes
Eigenvector failed to converge on strongly component (directed) (max_iter reached)
Eigenvector failed to converge on s

## Community detection
Run Louvain on the undirected graph (Set `run_gn_sample=True` to also run Girvan–Newman on sampled subgraph)

In [8]:
partition, num_comms, sizes = detect_communities(G, run_gn_sample=True)
communities_path = save_communities(partition)

print(f"Communities saved to: {communities_path}")
print(f"Total communities: {num_comms}")
print(f"Top community sizes: {sizes[:10]}")

Running Louvain community detection...
Detected 21389 communities via Louvain
Largest communities (by size): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Running Girvan–Newman on a sampled subgraph (top 1500 by degree)...
Girvan–Newman split produced 749 communities on the sample. Sizes: [13, 13, 12, 12, 10, 9, 9, 8, 8, 8, 8, 8, 7, 7, 7, 7, 7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,